In [1]:
from pprint import pprint
import numpy as np
import re
import nltk
import h5py
import io
import json
import JsonUtil
import tensorflow as tf
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector, merge, Flatten
from keras.layers import recurrent, Input, Bidirectional, LSTM, Lambda, GRU
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers
## from rnnlayer import Attention, SimpleAttention
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

c:\users\varun\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## loading the glove dictionary into the memory for usage in 
## making the word embeddings
GloveMatrix = {}
dimensionIndex = '300'
validationContext = [] 
validationQuestion = [] 
validationQuestion_id = [] 
validationToken2CharIdx = []
validationContextOriginal = []
maxLenvalidationContext = 0
maxLenvalidationQuestion = 0
trainContext = [] 
trainQuestion = [] 
trainQuestion_id = [] 
trainAnswerBegin = [] 
trainAnswerEnd = [] 
trainAnswerText = [] 
maxLenTrainingContext = 0
maxLenTrainingQuestion = 0
vocabulary = {}
contextMaxlen = 0
questionMax = 0

In [3]:
## function for loading the glove files into the memory it
## will be used for vectorizing the vocabulary that we will be
## creating
def getGloveDictionary(gloveFile):
    f = open(gloveFile,'r',encoding="utf8")
    for line in f:
        values = line.split()
        currWord = values[0]
        vectorizedNotation = values[1:]
        ## converting vectorized notation to array so that we 
        ## can directly use this as an arrya
        vector = np.asarray(vectorizedNotation, dtype='float32')
        GloveMatrix[currWord] = vector 
    f.close()

In [4]:
## tokenizing the line
## returning the tokenized line with the tokens
def tokenize(currLine):
    finalTokens = []
    tokens = re.split('(\W+)?', currLine)
    for currToken in tokens:
        if (currToken.strip()):
            finalTokens.append(currToken.strip())
    return finalTokens

In [5]:
def removeCommonData(data):
    data1 = data.replace("''", '" ')
    data1 = data1.replace("``", '" ')
    data = data.lower()
    return data

In [6]:
def processTrainingQues(qas, context, contextTokenized):
    global maxLenTrainingQuestion
    for qa in qas:
        question = qa._question_
        question = removeCommonData(question)
        questionTokenized = tokenize(question)
        if len(questionTokenized) > maxLenTrainingQuestion:
            maxLenTrainingQuestion = len(questionTokenized)
        question_id = qa._id_
        answers = qa._answers_
        for answer in answers:
            answerText = answer._text_
            answerTokenized = tokenize(answerText.lower())
            # find indices of beginning/ending words of answer span among tokenized context
            contextToAnswerFirstWord = context[:answer._answer_start_ + len(answerTokenized[0])]
            answerBeginIndex = len(tokenize(contextToAnswerFirstWord.lower())) - 1
            answerEndIndex = answerBeginIndex + len(answerTokenized) - 1
            trainQuestion.append(questionTokenized)
            trainQuestion_id.append(str(question_id))
            trainAnswerBegin.append(answerBeginIndex)
            trainAnswerEnd.append(answerEndIndex)
            trainAnswerText.append(answerText)
            trainContext.append(contextTokenized)

In [7]:
def splitDatasets(f):
    global maxLenTrainingContext
    for data in f:
        paragraphs = data._paragraphs_
        for paragraph in paragraphs:
            context = paragraph._context_
            context1 = context.replace("''", '" ')
            context1 = context1.replace("``", '" ')
            contextTokenized = tokenize(context.lower())
            contextLength = len(contextTokenized)
            if contextLength > maxLenTrainingContext:
                maxLenTrainingContext = contextLength
            qas = paragraph._qas_
            processTrainingQues(qas, context1, contextTokenized);

In [8]:
def getTokens(line):
    #tokens1 = nltk.word_tokenize(line)
    tokens = tokenize(line)
    ## removing bad datasets from the tokens
    for i in range(len(tokens)):
        tokens[i] = tokens[i].replace("''", '"')
        tokens[i].replace("``", '"')
    idx = 0;
    token_idx = 0;
    tokensStartIndex = [None]*len(tokens)   
    '''Code for getting the respective tokens and there respective index'''
    while idx < len(line) and token_idx < len(tokens):
        word = tokens[token_idx]
        if line[idx:idx+len(word)] == word:
            tokensStartIndex[token_idx] = idx
            idx = idx + len(word)
            token_idx = token_idx + 1 
        else:
            idx = idx + 1
    return tokens, tokensStartIndex

In [9]:
def processValidationQuestions(qas, tokenIdx2CharIdx, context, contextTokenized):
    global maxLenvalidationQuestion
    for qa in qas:
        question = qa._question_
        question = removeCommonData(question);
        questionTokenized = tokenize(question)
        if len(questionTokenized) > maxLenvalidationQuestion:
            maxLenvalidationQuestion = len(questionTokenized)
        question_id = qa._id_
        answers = qa._answers_
        validationQuestion.append(questionTokenized)
        validationQuestion_id.append(str(question_id))
        validationToken2CharIdx.append(tokenIdx2CharIdx)
        validationContextOriginal.append(context)
        validationContext.append(contextTokenized)

In [10]:
def splitValDatasets(f): 
    global maxLenvalidationContext
    for data in f:
        paragraphs = data._paragraphs_
        for paragraph in paragraphs:
            context = paragraph._context_
            context1 = context.replace("''", '" ')
            context1 = context1.replace("``", '" ')
            contextTokenized, tokenIdx2CharIdx = getTokens(context1.lower())
            contextLength = len(contextTokenized)
            if contextLength > maxLenvalidationContext:
                maxLenvalidationContext = contextLength
            qas = paragraph._qas_
            processValidationQuestions(qas, tokenIdx2CharIdx, context, contextTokenized);

In [11]:
def vectorizeTrainingData(dictionaryWordIndex, maximumcontext, maximumquestion):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(trainContext)):
        x=[]
        xq=[]
        for word in trainContext[i]:
            x.append(dictionaryWordIndex[word])
        for word in trainQuestion[i]:
            xq.append(dictionaryWordIndex[word])
        # map the first and last words of answer span to one-hot representations
        y_Begin =  np.zeros(len(trainContext[i]))
        y_Begin[trainAnswerBegin[i]] = 1
        y_End = np.zeros(len(trainContext[i]))
        y_End[trainAnswerEnd[i]] = 1
        X.append(x)
        Xq.append(xq)
        YBegin.append(y_Begin)
        YEnd.append(y_End)
    return pad_sequences(X, maxlen=maximumcontext, padding='post'),pad_sequences(Xq, maxlen=maximumquestion, padding='post'),pad_sequences(YBegin, maxlen=maximumcontext, padding='post'),pad_sequences(YEnd, maxlen=maximumcontext, padding='post')

In [12]:
def vectorizeValData(dictionaryWordIndex, maximumcontext, maximumquestion):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(validationContext)):
        x=[]
        xq=[]
        for word in validationContext[i]:
            x.append(dictionaryWordIndex[word])
        for word in validationQuestion[i]:
            xq.append(dictionaryWordIndex[word])
        X.append(x)
        Xq.append(xq)
    return pad_sequences(X, maxlen=maximumcontext, padding='post'), pad_sequences(Xq, maxlen=maximumquestion, padding='post')

In [13]:
def loadData():
    trainData = JsonUtil.import_qas_data('data/train-v1.1.json')
    splitDatasets(trainData)
    valData = JsonUtil.import_qas_data('data/dev-v1.1.json')
    splitValDatasets(valData)

In [14]:
'''Change the below function correctly'''
def getVocab():
    vocabulary = {}
    global maxLenTContext
    global maxLenVContext
    for words in trainContext+trainQuestion+validationContext+validationQuestion:
        for word in words:
            if word not in vocabulary:
                vocabulary[word] = 1
#     for words in trainQuestion:
#         for word in words:
#             if word in vocabulary:
#                 continue
#             else:
#                 vocabulary[word] = 1
#     for words in validationContext:
#         for word in words:
#             if word in vocabulary:
#                 continue
#             else:
#                 vocabulary[word] = 1
#     for words in validationQuestion:
#         for word in words:
#             if word not in vocabulary:
#                 vocabulary[word] = 1
    vocabulary = sorted(vocabulary.keys())  
#     vocab_size = len(vocabulary) + 1
    dictionaryWordIndex = {}
    for i in range(len(vocabulary)):
        dictionaryWordIndex[vocabulary[i]]=i+1
    maximumcontext = 0
    maximumquestion = 0
    if (maxLenTrainingContext > maxLenvalidationContext):
        maximumcontext = maxLenTrainingContext
    else:
        maximumcontext = maxLenvalidationContext    
    if(maxLenvalidationQuestion > maxLenTrainingQuestion):
        maximumquestion = maxLenvalidationQuestion
    else:
        maximumquestion = maxLenTrainingQuestion
    return vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion

In [15]:
def getVectorizedData(vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion):    
    tX, tXq, tYBegin, tYEnd = vectorizeTrainingData(dictionaryWordIndex, maximumcontext, maximumquestion)
    randindex = np.random.permutation(tX.shape[0])
    tX = tX[randindex, :]
    tXq = tXq[randindex, :]
    tYBegin = tYBegin[randindex, :]
    tYEnd = tYEnd[randindex, :]
    vX, vXq = vectorizeValData(dictionaryWordIndex, maximumcontext, maximumquestion)
    return tX, tXq, tYBegin, tYEnd ,vX, vXq

In [16]:
def createEmbeddingMatrix(dictionary):
    
    EmbedMatrix = np.zeros((len(dictionary), int(dimensionIndex)))
    for word, i in dictionary.items():
        EmbedV = GloveMatrix.get(word)
        if EmbedV is not None:
            EmbedMatrix[i] = EmbedV
    return EmbedMatrix

In [17]:
def main():
    glovePath = 'data/glove.6B/glove.6B.' + str(dimensionIndex) + 'd.txt';
    getGloveDictionary(glovePath);
    loadData()
    vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion = getVocab()
    tX, tXq, tYBegin, tYEnd ,vX, vXq = getVectorizedData(vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion)
    EmbedMatrix = createEmbeddingMatrix(dictionaryWordIndex)
    embedding_layer = Embedding(len(dictionaryWordIndex),
                            int(dimensionIndex),
                            weights=[EmbedMatrix],
                            input_length=maximumcontext,
                            trainable=True)
    print("maximumcontext",maximumcontext)
    print("maximumquestion",maximumquestion)
    cinput = Input(shape=(maximumcontext,), dtype='int32', name='cinput')
    cembedding=embedding_layer(cinput)
    qinput = Input(shape=(maximumquestion,), dtype='int32', name='qinput')
    qembedding=embedding_layer(qinput)
    # BiDirectional LSTM Layer
    D = Bidirectional(LSTM(150, return_sequences=False))(cembedding)
    # print(D.shape)
    Q = Bidirectional(LSTM(150, return_sequences=False))(qembedding)
    # print(Q.shape)
    L = merge([D, Q], mode='concat')
    ## Code for the Attention Layer 
    # print(qembedding.shape)
    # print(qembedding.shape)
    # Q = Bidirectional(LSTM(100, return_sequences=True))(qembedding)
    # Q1 = Dropout(0.2)(Q)
    # print(Q.shape)
    # clstm1 = Attention(100, Q1, 200, return_sequences=True, name='clstm1')(cembedding)
    # qlstm2 = Attention(100, clstm1, 100, return_sequences=True, name='qlstm2')(Q1)
    # clstm2 = Attention(100, qlstm2, 100, return_sequences=True, name='clstm2')(clstm1)
    # qlstm4 = Bidirectional(LSTM(100, name='qlstm4'))(qlstm3)
    # h = merge([qlstm4, clstm3], mode='concat', name='merge1'
    answerPtrBegin_output = Dense(maximumcontext, activation='softmax')(L)
    Lmerge = merge([L, answerPtrBegin_output], mode='concat', name='merge2')
    answerPtrEnd_output = Dense(maximumcontext, activation='softmax')(Lmerge)
    model = Model(input=[cinput, qinput], output=[answerPtrBegin_output, answerPtrEnd_output])
    adam = optimizers.Adam()#, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy',
                  loss_weights=[.04, 0.04], metrics=['accuracy'])
    model.summary()
    filepath1="weights/weights-matchlstm-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
    checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')
#     checkpoint2 = EarlyStopping(monitor='val_dense_1_loss', min_delta=0.01, patience=3, verbose=0, mode='auto')
    callbacks_list = [checkpoint1] #checkpoint2 
    model.fit([tX, tXq], [tYBegin, tYEnd], epochs=10, batch_size=128, shuffle=True, validation_split=0.25, \
              callbacks=callbacks_list)
    predictions = model.predict([vX, vXq], batch_size=128)
    ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
    ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
    for i in range(predictions[0].shape[0]):
        ansBegin[i] = predictions[0][i, :].argmax()
        ansEnd[i] = predictions[1][i, :].argmax()
    answers = {}
    for i in range(len(validationQuestion_id)):
        #print i
        if ansBegin[i] >= len(validationContext[i]):
            answers[validationQuestion_id[i]] = ""
        elif ansEnd[i] >= len(validationContext[i]):
            answers[validationQuestion_id[i]] = validationContextOriginal[i][validationToken2CharIdx[i][ansBegin[i]]:]
        else:
            answers[validationQuestion_id[i]] = validationContextOriginal[i][validationToken2CharIdx[i][ansBegin[i]]:validationToken2CharIdx[i][ansEnd[i]]+len(validationContext[i][ansEnd[i]])]
    with io.open('predictedanswer.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(answers, ensure_ascii=False))

In [18]:
main()

c:\users\varun\anaconda3\envs\tensorflow\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


maximumcontext 844
maximumquestion 60


c:\users\varun\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
c:\users\varun\anaconda3\envs\tensorflow\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
c:\users\varun\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
c:\users\varun\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cinput (InputLayer)             (None, 844)          0                                            
__________________________________________________________________________________________________
qinput (InputLayer)             (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 844, 300)     26538300    cinput[0][0]                     
                                                                 qinput[0][0]                     
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 300)          541200      embedding_1[0][0]                
__________

InternalError: Blas GEMM launch failed : a.shape=(128, 150), b.shape=(150, 150), m=128, n=150, k=150
	 [[Node: bidirectional_1/while/MatMul_7 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](bidirectional_1/while/Switch_2:1, bidirectional_1/while/MatMul_7/Enter)]]
	 [[Node: metrics/acc_1/Mean/_267 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6611_metrics/acc_1/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'bidirectional_1/while/MatMul_7', defined at:
  File "c:\users\varun\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\varun\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\varun\anaconda3